# Libraries

In [ ]:
import json
import pandas as pd

from nltk.tokenize import RegexpTokenizer

import gc
from thefuzz import fuzz
import numpy as np
from tqdm.auto import tqdm

from catboost import CatBoostClassifier, Pool

import warnings

In [ ]:
pd.set_option('display.max_columns', 256)
warnings.filterwarnings('ignore')

# Data

In [ ]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
rtrain_pairs = train_pairs.copy()
rtrain_pairs.variantid1, rtrain_pairs.variantid2 = rtrain_pairs.variantid2, rtrain_pairs.variantid1
train_pairs

In [ ]:
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
rtest_pairs = test_pairs.copy()
rtest_pairs.variantid1, rtest_pairs.variantid2 = rtest_pairs.variantid2, rtest_pairs.variantid1
test_pairs

In [ ]:
train_data = pd.read_parquet('./datasets/train_data.parquet').set_index('variantid')
train_data['characteristic_attributes_mapping'] = train_data['characteristic_attributes_mapping'].fillna('{}').apply(lambda x: json.loads(x))
train_data['categories'] = train_data['categories'].apply(lambda x: json.loads(x))
train_data['main_pic_embeddings_resnet_v1'] = train_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
train_data = pd.concat([train_data, pd.read_parquet('./datasets/name_labse_embs_train.parquet').set_index('variantid')], axis=1)
train_data = pd.concat([train_data, pd.read_pickle('./datasets/neuro_embeds_train.pickle')], axis=1)
train_data = pd.concat([train_data, pd.read_pickle('./datasets/labse_tuned_train.pickle')], axis=1)
train_data = pd.concat([train_data, pd.read_pickle('./datasets/multimodal_tuned_train.pickle')], axis=1)
train_data

In [ ]:
test_data = pd.read_parquet('./datasets/test_data.parquet').set_index('variantid')
test_data['characteristic_attributes_mapping'] = test_data['characteristic_attributes_mapping'].fillna('{}').apply(lambda x: json.loads(x))
test_data['categories'] = test_data['categories'].apply(lambda x: json.loads(x))
test_data['main_pic_embeddings_resnet_v1'] = test_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
test_data = pd.concat([test_data, pd.read_parquet('./datasets/name_labse_embs_test.parquet').set_index('variantid')], axis=1)
test_data = pd.concat([test_data, pd.read_pickle('./datasets/neuro_embeds_test.pickle')], axis=1)
test_data = pd.concat([test_data, pd.read_pickle('./datasets/labse_tuned_test.pickle')], axis=1)
test_data = pd.concat([test_data, pd.read_pickle('./datasets/multimodal_tuned_test.pickle')], axis=1)
test_data

In [ ]:
train_df = pd.read_pickle('./dataframes/train_df.pickle')
rtrain_df = pd.read_pickle('./dataframes/rtrain_df.pickle')
test_df = pd.read_pickle('./dataframes/test_df.pickle')
rtest_df = pd.read_pickle('./dataframes/rtest_df.pickle')

In [ ]:
cat_features = [
    'cat3', 'cat4',
    'Страна-изготовитель_1', 'Страна-изготовитель_2', 
    'Оперативная память_1', 'Оперативная память_2',
    'Бренд процессора_1', 'Бренд процессора_2',
    'Модуль связи Bluetooth_1', 'Модуль связи Bluetooth_2',
    'Назначение_1', 'Назначение_2',
]
embedding_features = ['main_pic_1', 'main_pic_2', 'name_bert_1', 'name_bert_2', 'name_labse_1', 'name_labse_2']

In [ ]:
train_pool = Pool(
    data=train_df.drop(['target']+embedding_features, axis = 1),
    cat_features=cat_features,
)
rtrain_pool = Pool(
    data=rtrain_df.drop(['target']+embedding_features, axis = 1),
    cat_features=cat_features,
)
test_pool = Pool(
    data=test_df.drop(embedding_features, axis = 1),
    cat_features=cat_features,
)
rtest_pool = Pool(
    data=rtest_df.drop(embedding_features, axis = 1),
    cat_features=cat_features,
)

In [ ]:
model_cb_pseudo = CatBoostClassifier().load_model('./models/golden_model_cb_pseudo.cbm')

In [ ]:
train_pairs['cb_pseudo_pred'] = rtrain_pairs['cb_pseudo_pred'] = \
(model_cb_pseudo.predict_proba(train_pool)[:, 1] + model_cb_pseudo.predict_proba(rtrain_pool)[:, 1]) / 2.

test_pairs['cb_pseudo_pred'] = rtest_pairs['cb_pseudo_pred'] = \
(model_cb_pseudo.predict_proba(test_pool)[:, 1] + model_cb_pseudo.predict_proba(rtest_pool)[:, 1]) / 2.

In [ ]:
train_pool = Pool(
    data=train_df.drop(['target'], axis = 1),
    cat_features=cat_features,
    embedding_features=embedding_features
)
rtrain_pool = Pool(
    data=rtrain_df.drop(['target'], axis = 1),
    cat_features=cat_features,
    embedding_features=embedding_features
)
test_pool = Pool(
    data=test_df,
    cat_features=cat_features,
    embedding_features=embedding_features
)
rtest_pool = Pool(
    data=rtest_df,
    cat_features=cat_features,
    embedding_features=embedding_features
)

In [ ]:
model_cb = CatBoostClassifier().load_model('./models/golden_model_cb.cbm')

In [ ]:
%%time
train_pairs['cb_pred'] = rtrain_pairs['cb_pred'] = \
(model_cb.predict_proba(train_pool)[:, 1] + model_cb.predict_proba(rtrain_pool)[:, 1]) / 2.

test_pairs['cb_pred'] = rtest_pairs['cb_pred'] = \
(model_cb.predict_proba(test_pool)[:, 1] + model_cb.predict_proba(rtest_pool)[:, 1]) / 2.

In [ ]:
prev_cols = []
for col in train_df.columns:
    if col == 'target':
        continue
    if col.startswith('diff_'):
        break
    prev_cols.append(col)
len(prev_cols)

In [ ]:
train_pairs = pd.concat([train_pairs, train_df[prev_cols]], axis=1)
rtrain_pairs = pd.concat([rtrain_pairs, rtrain_df[prev_cols]], axis=1)
test_pairs = pd.concat([test_pairs, test_df[prev_cols]], axis=1)
rtest_pairs = pd.concat([rtest_pairs, rtest_df[prev_cols]], axis=1)

In [ ]:
train_pairs = train_pairs[pd.read_csv('./datasets/val_idx.csv', index_col=0).values].copy()
rtrain_pairs = rtrain_pairs[pd.read_csv('./datasets/val_idx.csv', index_col=0).values].copy()

In [ ]:
idx = list(train_pairs.index)

In [ ]:
from sklearn.model_selection import train_test_split

idx_train, idx_val = train_test_split(idx, test_size=0.2, random_state=56)
len(idx_train), len(idx_val)

In [ ]:
val_pairs = train_pairs.loc[idx_val].copy()
rval_pairs = rtrain_pairs.loc[idx_val].copy()
train_pairs = train_pairs.loc[idx_train].copy()
rtrain_pairs = rtrain_pairs.loc[idx_train].copy()

In [ ]:
train_pairs

In [ ]:
val_pairs

In [ ]:
test_pairs

# Features

In [ ]:
colors_mapper = {
 'ярко-синий': 'ярко-синий',
 'ярко-розовый': 'ярко-розовый',
 'ярко-зеленый': 'ярко-зеленый',
 'ярко-желтый': 'ярко-желтый',
 'янтарный': 'янтарный',
 'электрик': 'электрик',
 'экрю': 'экрю',
 'шоколадный': 'шоколадный',
 'черный': 'черный',
 'черно-синий': 'черно-синий',
 'черно-серый': 'черно-серый',
 'черно-красный': 'черно-красный',
 'черно-зеленый': 'черно-зеленый',
 'черн': 'черный',
 'чер': 'черный',
 'циан': 'бирюзовый',
 'цементный': 'цементный',
 'хаки': 'хаки',
 'фуксия': 'фуксия',
 'фисташковый': 'фисташковый',
 'фиолетовый': 'фиолетовый',
 'фиолетово-синий': 'фиолетово-синий',
 'фиолет': 'фиолетовый',
 'фиол': 'фиолетовый',
 'фиалковый': 'фиалковый',
 'тыквенный': 'тыквенный',
 'тыква': 'тыквенный',
 'травяной': 'травяной',
 'томатный': 'томатный',
 'тиффани': 'тиффани',
 'терракотовый': 'терракотовый',
 'терракота': 'терракотовый',
 'темно-фиолетовый': 'темно-фиолетовый',
 'темно-синий': 'темно-синий',
 'темно-серый': 'темно-серый',
 'темно-розовый': 'темно-розовый',
 'темно-оранжевый': 'темно-оранжевый',
 'темно-оливковый': 'темно-оливковый',
 'темно-красный': 'темно-красный',
 'темно-коричневый': 'темно-коричневый',
 'темно-зеленый': 'темно-зеленый',
 'темно-голубой': 'темно-голубой',
 'темно-бирюзовый': 'темно-бирюзовый',
 'темно-бежевый': 'темно-бежевый',
 'сливовый': 'сливовый',
 'сиреневый': 'сиреневый',
 'синий': 'синий',
 'сине-зеленый': 'сине-зеленый',
 'син': 'синий',
 'серый': 'серый',
 'серовато-зеленый': 'серовато-зеленый',
 'серо-коричневый': 'серо-коричневый',
 'серо-зеленый': 'серо-зеленый',
 'серо-голубой': 'серо-голубой',
 'серо-бежевый': 'серо-бежевый',
 'серебряный': 'серебряный',
 'серебристый': 'серебристый',
 'серебристо-серый': 'серебристо-серый',
 'сер': 'серый',
 'сепия': 'сепия',
 'светло-фиолетовый': 'светло-фиолетовый',
 'светло-синий': 'светло-синий',
 'светло-серый': 'светло-серый',
 'светло-розовый': 'светло-розовый',
 'светло-пурпурный': 'светло-пурпурный',
 'светло-коричневый': 'светло-коричневый',
 'светло-золотистый': 'светло-золотистый',
 'светло-зеленый': 'светло-зеленый',
 'светло-желтый': 'светло-желтый',
 'светло-голубой': 'светло-голубой',
 'светло-бирюзовый': 'светло-бирюзовый',
 'светло-бежевый': 'светло-бежевый',
 'сапфировый': 'сапфировый',
 'салатовый': 'салатовый',
 'рыжий': 'рыжий',
 'розовый': 'розовый',
 'розово-фиолетовый': 'розово-фиолетовый',
 'розово-золотой': 'розово-золотой',
 'разноцветный': 'разноцветный',
 'пурпурный': 'пурпурный',
 'пурпурно-фиолетовый': 'пурпурно-фиолетовый',
 'песочный': 'песочный',
 'перу': 'перу',
 'персиковый': 'персиковый',
 'охра': 'охра',
 'орхидея': 'орхидея',
 'оранжевый': 'оранжевый',
 'оранжево-розовый': 'оранжево-розовый',
 'оливковый': 'оливковый',
 'огненно-красный': 'огненно-красный',
 'нефритовый': 'нефритовый',
 'небесный': 'небесный',
 'мятный': 'мятный',
 'мятно-зеленый': 'мятно-зеленый',
 'мята': 'мятный',
 'мультиколор': 'мультиколор',
 'морковный': 'морковный',
 'молочный': 'молочный',
 'многоцветный': 'многоцветный',
 'медный': 'медный',
 'марсала': 'марсала',
 'малиновый': 'малиновый',
 'малиново-красный': 'малиново-красный',
 'малахитовый': 'малахитовый',
 'льняной': 'льняной',
 'лимонный': 'лимонный',
 'лиловый': 'лиловый',
 'латунный': 'латунный',
 'лаймовый': 'лаймовый',
 'лайм': 'лаймовый',
 'лазурный': 'лазурный',
 'лавандовый': 'лавандовый',
 'лаванда': 'лавандовый',
 'кремовый': 'кремовый',
 'красный': 'красный',
 'красновато-коричневый': 'красновато-коричневый',
 'красно-оранжевый': 'красно-оранжевый',
 'красно-коричневый': 'красно-коричневый',
 'красн': 'красный',
 'крас': 'красный',
 'кофейный': 'кофейный',
 'космос': 'космос',
 'коричневый': 'коричневый',
 'коричнево-красный': 'коричнево-красный',
 'коричнево-бежевый': 'коричнево-бежевый',
 'коралловый': 'коралловый',
 'кораллово-красный': 'кораллово-красный',
 'кобальтовый': 'кобальтовый',
 'кирпичный': 'кирпичный',
 'кирпично-красный': 'кирпично-красный',
 'кварцевый': 'кварцевый',
 'кардинал': 'кардинал',
 'канареечный': 'канареечный',
 'камуфляжный': 'камуфляжный',
 'индиго': 'индиго',
 'изумрудный': 'изумрудный',
 'изумрудно-зеленый': 'изумрудно-зеленый',
 'изумруд': 'изумрудный',
 'золотой': 'золотой',
 'золотистый': 'золотистый',
 'зеленый': 'зеленый',
 'зелено-серый': 'зелено-серый',
 'зел': 'зеленый',
 'жемчужно-белый': 'жемчужно-белый',
 'желтый': 'желтый',
 'желто-розовый': 'желто-розовый',
 'желто-зеленый': 'желто-зеленый',
 'желт': 'желтый',
 'гусеница': 'гусеница',
 'грушевый': 'грушевый',
 'графит': 'графит',
 'гранитный': 'гранитный',
 'гранатовый': 'гранатовый',
 'горчичный': 'горчичный',
 'голубой': 'голубой',
 'голуб': 'голубой',
 'глициния': 'глициния',
 'вишня': 'вишневый',
 'вишневый': 'вишневый',
 'васильковый': 'васильковый',
 'ванильный': 'ванильный',
 'бурый': 'бурый',
 'бронзовый': 'бронзовый',
 'бордовый': 'бордовый',
 'бордо': 'бордовый',
 'болотный': 'болотный',
 'бледно-розовый': 'бледно-розовый',
 'бледно-пурпурный': 'бледно-пурпурный',
 'бледно-желтый': 'бледно-желтый',
 'бирюзовый': 'бирюзовый',
 'бирюзово-зеленый': 'бирюзово-зеленый',
 'белый': 'белый',
 'белоснежный': 'белоснежный',
 'бело-зеленый': 'бело-зеленый',
 'бел': 'белый',
 'бежевый': 'бежевый',
 'бежево-серый': 'бежево-серый',
 'бежево-розовый': 'бежево-розовый',
 'баклажановый': 'баклажановый',
 'антрацитовый': 'антрацитовый',
 'аметистовый': 'аметистовый',
 'алый': 'алый',
 'аквамариновый': 'аквамариновый',
 'аква': 'аква',
 'абрикосовый': 'абрикосовый',
 'yellow': 'желтый',
 'wine': 'wine',
 'white': 'белый',
 'violet': 'фиолетовый',
 'vanilla': 'ванильный',
 'ultramarine': 'ultramarine',
 'turquoise': 'бирюзовый',
 'tomato': 'томатный',
 'teal': 'teal',
 'tan': 'tan',
 'snow': 'snow',
 'silver': 'серебряный',
 'sapphire': 'сапфировый',
 'red': 'красный',
 'purple': 'фиолетовый',
 'pink': 'розовый',
 'peru': 'перу',
 'pear': 'грушевый',
 'peach': 'персиковый',
 'orchid': 'орхидея',
 'orange': 'оранжевый',
 'olive': 'оливковый',
 'navy': 'navy',
 'magenta': 'пурпурный',
 'linen': 'linen',
 'lime': 'лаймовый',
 'lilac': 'сиреневый',
 'lemon': 'lemon',
 'lavender': 'лавандовый',
 'khaki': 'хаки',
 'jade': 'нефритовый',
 'ivory': 'ivory',
 'indigo': 'индиго',
 'grey': 'серый',
 'green': 'зеленый',
 'gray': 'серый',
 'gold': 'золотой',
 'fuchsia': 'фуксия',
 'flax': 'flax',
 'emerald': 'emerald',
 'denim': 'denim',
 'cyan': 'бирюзовый',
 'cream': 'кремовый',
 'corn': 'corn',
 'coral': 'коралловый',
 'copper': 'медный',
 'cobalt': 'кобальтовый',
 'chocolate': 'шоколадный',
 'burgundy': 'бордовый',
 'buff': 'buff',
 'brown': 'коричневый',
 'bronze': 'бронзовый',
 'brass': 'латунный',
 'blue': 'голубой',
 'blond': 'blond',
 'black': 'черный',
 'beige': 'бежевый',
 'azure': 'лазурный',
 'aquamarine': 'аквамариновый',
 'aqua': 'аквамариновый',
 'amethyst': 'аметистовый',
 'amber': 'янтарный'
}

In [ ]:
clar_tokenizer = RegexpTokenizer(r"\([^()]+\)")

In [ ]:
def calc_dists(df, prefix, embs_1, embs_2):
    l1_dists, l2_dists, cos_dists, inf_dists = [], [], [], []
    for emb_1, emb_2 in zip(embs_1, embs_2):
        len_1 = (emb_1**2).sum()**0.5
        len_2 = (emb_2**2).sum()**0.5
        l1_dists.append(
            np.abs(emb_1 - emb_2).sum()
        )
        l2_dists.append(
            ((emb_1 - emb_2)**2).sum()**0.5
        )
        cos_dists.append(
            (emb_1 @ emb_2) / len_1 / len_2
        )
        inf_dists.append(
            np.abs(emb_1 - emb_2).max()
        )
    df[f'{prefix}_l1_dist'] = l1_dists
    df[f'{prefix}_l2_dist'] = l2_dists
    df[f'{prefix}_cos_dist'] = cos_dists

In [ ]:
def make_features(pairs, data):
    gc.collect()
    df = pairs.copy()   
    df['cb_mean_pred'] = (df['cb_pseudo_pred'] + df['cb_pred']) / 2
    
    # multi modal
    multimodal_tuned_1 = data.loc[df.variantid1, 'multimodal_tuned_768'].values
    multimodal_tuned_2 = data.loc[df.variantid2, 'multimodal_tuned_768'].values
    calc_dists(
        df, 'multimodal_tuned', 
        multimodal_tuned_1,
        multimodal_tuned_2
    )
    
    # labse
    labse_tuned_1 = data.loc[df.variantid1, 'labse_tuned_768'].values
    labse_tuned_2 = data.loc[df.variantid2, 'labse_tuned_768'].values
    calc_dists(
        df, 'labse_tuned', 
        labse_tuned_1,
        labse_tuned_2
    )
    
    # fix
    names_1 = data.loc[pairs.variantid1, 'name']
    names_2 = data.loc[pairs.variantid2, 'name']
    colors_1 = data.loc[pairs.variantid1, 'color_parsed']
    colors_2 = data.loc[pairs.variantid2, 'color_parsed']
    
    same_colors = []
    all_colors = []
    for color_1, color_2, name_1, name_2 in tqdm(zip(colors_1, colors_2, names_1, names_2), total=len(df)):
        if color_1 is None:
            color_1 = []
        if color_2 is None:
            color_2 = []
            
        color_1 = set([colors_mapper[c] for c in color_1])
        color_2 = set([colors_mapper[c] for c in color_2])
        for color in colors_mapper:
            if color + ' ' in name_1 or ' ' + color in name_1:
                color_1.add(colors_mapper[color])
            if color + ' ' in name_2 or ' ' + color in name_2:
                color_2.add(colors_mapper[color])
        
        same_colors.append(
            len(color_1 & color_2)
        )
        all_colors.append(
            len(color_1 | color_2)
        )
    df['same_colors'] = same_colors
    df['all_colors'] = all_colors
    df['iou_colors'] = df['same_colors'] / df['all_colors']
    df.loc[df['all_colors']==0, 'iou_colors'] = 0
    df['not_same_colors'] = df['all_colors'] - df['same_colors']
    
    clars_set_dist = []
    for name_1, name_2 in zip(names_1, names_2): 
        clars_1 = clar_tokenizer.tokenize(name_1)
        clars_2 = clar_tokenizer.tokenize(name_2)
        if len(clars_1) == 0 or len(clars_2) == 0:
            clars_set_dist.append(100 * (1 + (len(clars_1) == 0) + (len(clars_2) == 0)))
        else:
            clars_1 = [v[1:-1] for v in clars_1]
            clars_2 = [v[1:-1] for v in clars_2]
            clars_set_dist.append(
                fuzz.token_set_ratio(clars_1, clars_2)
            )
    df['clars_set_dist'] = clars_set_dist
        
    # embeddings    
    df['multimodal_tuned_1'] = list(data.loc[pairs.variantid1, 'multimodal_tuned_768'])
    df['multimodal_tuned_2'] = list(data.loc[pairs.variantid2, 'multimodal_tuned_768'])
    df['labse_tuned_tuned_1'] = list(data.loc[pairs.variantid1, 'labse_tuned_768'])
    df['labse_tuned_tuned_2'] = list(data.loc[pairs.variantid2, 'labse_tuned_768'])
    
    return df.drop(['variantid1', 'variantid2'], axis=1)

In [ ]:
%%time
train_df = make_features(train_pairs, train_data)
train_df

In [ ]:
rtrain_df = make_features(rtrain_pairs, train_data)
val_df = make_features(val_pairs, train_data)
rval_df = make_features(rval_pairs, train_data)

# Model

In [ ]:
all_train = pd.concat([train_df, rtrain_df], axis=0)
all_val = pd.concat([val_df, rval_df], axis=0)
all_train

In [ ]:
cat_features = ['cat3', 'cat4']
embedding_features = ['multimodal_tuned_1', 'multimodal_tuned_2', 'labse_tuned_tuned_1', 'labse_tuned_tuned_2']

In [ ]:
train_pool = Pool(
    data=all_train.drop('target', axis = 1),
    label=all_train['target'].values,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=all_train['cb_pseudo_pred']
)

val_pool = Pool(
    data=all_val.drop('target', axis = 1),
    label=all_val['target'].values,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=all_val['cb_pseudo_pred']
)

In [ ]:
params = {
    'loss_function': 'CrossEntropy',
    'eval_metric': 'PRAUC',
    'task_type': 'CPU',
    'max_depth': 8,
    'learning_rate': 0.04,
    'iterations': 2000
}

In [ ]:
%%time
model_cb = CatBoostClassifier(**params, random_seed=56, cat_features=cat_features, embedding_features=embedding_features)
model_cb.fit(train_pool, eval_set=val_pool, verbose=250, plot=True, use_best_model=True, early_stopping_rounds=300)

In [ ]:
np.max(model_cb.get_evals_result()['validation']['PRAUC'])

In [ ]:
imp = model_cb.get_feature_importance(prettified=True)
imp.head(10)

In [ ]:
pd.concat([val_pairs, rval_pairs], axis=0)[all_val.target != model_cb.predict(val_pool)]

# Inference

In [ ]:
test_df = make_features(test_pairs, test_data)
rtest_df = make_features(rtest_pairs, test_data)

In [ ]:
test_pool = Pool(
    data=test_df,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=test_df['cb_pseudo_pred']
)

rtest_pool = Pool(
    data=rtest_df,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=rtest_df['cb_pseudo_pred']
)

In [ ]:
preds = model_cb.predict_proba(test_pool)[:,1]
preds

In [ ]:
rpreds = model_cb.predict_proba(rtest_pool)[:,1]
rpreds

In [ ]:
final_preds = (preds + rpreds) / 2.
final_preds

In [ ]:
test_pairs['target'] = final_preds
test_pairs.to_csv('./golden_submit_ens.csv', index=False)
test_pairs.drop('target', axis=1, inplace=True)

In [ ]:
val_pool = Pool(
    data=all_val.drop('target', axis = 1),
    label=all_val['target'],
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=all_val['cb_pseudo_pred']
)

init_target = model_cb.predict_proba(val_pool)[:, 1]
mean_target = init_target.copy()
mean_target[:len(val_df)] += init_target[len(val_df):]
mean_target[len(val_df):] += init_target[:len(val_df)]
mean_target /= 2.
pseudo_all_val = all_val.copy().drop('target', axis=1)
pseudo_all_val['target'] = mean_target

In [ ]:
pseudo_test = test_df.copy()
pseudo_test['target'] = final_preds
pseudo_rtest = rtest_df.copy()
pseudo_rtest['target'] = final_preds

In [ ]:
pseudo_all_train = pd.concat([all_train, pseudo_all_val, pseudo_test, pseudo_rtest], axis=0)
pseudo_all_train

In [ ]:
train_pool = Pool(
    data=pseudo_all_train.drop(['target']+embedding_features, axis = 1),
    label=pseudo_all_train['target'],
    cat_features=cat_features,
    baseline=pseudo_all_train['cb_pseudo_pred']
)

val_pool = Pool(
    data=all_val.drop(['target']+embedding_features, axis = 1),
    label=all_val['target'],
    cat_features=cat_features,
    baseline=all_val['cb_pseudo_pred']
)

In [ ]:
params = {
    'loss_function': 'CrossEntropy',
    'eval_metric': 'PRAUC',
    'task_type': 'CPU',
    'max_depth': 8,
    'learning_rate': 0.04,
    'iterations': 2000
}

In [ ]:
%%time
model_cb_pseudo = CatBoostClassifier(**params, random_seed=56, cat_features=cat_features, train_dir='./1/')
model_cb_pseudo.fit(train_pool, eval_set=val_pool, verbose=250, plot=True, use_best_model=True, early_stopping_rounds=300)

In [ ]:
np.max(model_cb_pseudo.get_evals_result()['validation']['PRAUC']) # 0.9127931864562437

In [ ]:
model_cb_pseudo.get_feature_importance(prettified=True).head(10)

In [ ]:
models = []
for seed in tqdm((7, 13, 19, 23, 31, 56)):
    model = CatBoostClassifier(**params, random_seed=seed, cat_features=cat_features, train_dir='./1/')
    model.fit(train_pool, eval_set=val_pool, verbose=250, plot=True, use_best_model=True, early_stopping_rounds=300)
    models.append(model)

In [ ]:
test_pool = Pool(
    data=test_df.drop(embedding_features, axis=1),
    cat_features=cat_features,
    baseline=test_df['cb_pseudo_pred']
)

In [ ]:
rtest_pool = Pool(
    data=rtest_df.drop(embedding_features, axis=1),
    cat_features=cat_features,
    baseline=rtest_df['cb_pseudo_pred']
)

In [ ]:
preds = model_cb_pseudo.predict_proba(test_pool)[:,1]
preds

In [ ]:
rpreds = model_cb_pseudo.predict_proba(rtest_pool)[:,1]
rpreds

In [ ]:
final_preds = (preds + rpreds) / 2.
final_preds

In [ ]:
test_pairs['target'] = final_preds
test_pairs.to_csv('./golden_submit_ens_pseudo.csv', index=False)
test_pairs.drop('target', axis=1, inplace=True)

In [ ]:
preds = 0
rpreds = 0
for model in models:
    preds += model.predict_proba(test_pool)[:,1]
    rpreds += model.predict_proba(rtest_pool)[:,1]
preds /= len(models)    
rpreds /= len(models)    

In [ ]:
final_preds = (preds + rpreds) / 2.
final_preds

In [ ]:
test_pairs['target'] = final_preds
test_pairs.to_csv('./golden_submit_ens_pseudo_seeds.csv', index=False)
test_pairs.drop('target', axis=1, inplace=True)